## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Mobile

### Links: 
___

In [7]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.citizen_payments_st_mobile
    (
        `report_date` Date,
        `citizen_id`  Int32,
        `state` String,
        `amount` Int32,
        `paid_at` DateTime,
        `refunded_at` DateTime,
        `refunded_amount` Int16,
        `from` String,
        `payment_id` Int32,
        `product_ids` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/citizen_payments_st_mobile/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.citizen_payments_st_mobile already exists. (TABLE_ALREADY_EXISTS) (version 25.3.3.42 (official build))


In [ ]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.citizen_payments_st_mobile_ch
    (
        `report_date` Date,
        `citizen_id`  Int32,
        `state` String,
        `amount` Int32,
        `paid_at` DateTime,
        `refunded_at` DateTime,
        `refunded_amount` Int16,
        `from` String,
        `payment_id` Int32,
        `product_ids` String
    )
    ENGINE = MergeTree()
    ORDER BY citizen_id
    """

ch.query_run(query_text)

In [ ]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.citizen_payments_st_mobile_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.citizen_payments_st_mobile_ch AS
    SELECT
        *
    FROM db1.citizen_payments_st_mobile
    """

ch.query_run(query_text)

___
## Tools
___
### query

In [10]:
query_text = """--sql
    SELECT
        *
    FROM db1.citizen_payments_st_mobile_ch
    order by report_date desc
    LIMIT 2
    """

ch.query_run(query_text)

report_date,citizen_id,state,amount,paid_at,refunded_at,refunded_amount,from,payment_id,product_ids
date,i64,str,i64,datetime[ns],datetime[ns],i64,str,i64,str
2025-07-28,32954,"""canceled""",85,1970-01-01 00:00:00,1970-01-01 00:00:00,0,"""yookassa""",2093222,"""[""standard""]"""
2025-07-28,33719,"""success""",69,2025-07-28 03:50:11,1970-01-01 00:00:00,0,"""yookassa""",2092304,"""[""standard""]"""


### Drop ch

In [ ]:
query_text = """
    DROP TABLE db1.citizen_payments_st_mobile_ch
    """
ch.query_run(query_text)

### Drop mv

In [ ]:
query_text = """
    DROP TABLE db1.citizen_payments_st_mobile_mv
    """
ch.query_run(query_text)

### refresh mv

In [8]:
query_text = """
SYSTEM REFRESH VIEW db1.citizen_payments_st_mobile_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
749697,"""43b0e46e-99cb-4d67-a0bb-ae2222…"
